### 어텐션 이해하기

Attention은 Query와 비슷한 값을 가진 Key를 찾아서 그 value를 얻는 과정입니다.

여기서는 흔히 json같이 널리 사용되는 key-value 방식과 비교하면서 설명하겠습니다.

### key-value 함수

Key-value 자료형은 파이썬에서는 *dictionary*라고 부르는 자료형입니다. 파이썬에서는 다음과 같이 사용할 수 있습니다.

```python
dic = {"computer": 9, "dog": 2, "cat": 3}
```

이와 같이 key와 value에 해당하는 값들을 넣고 key를 통해 value 값에 접근할 수 있습니다. 다시 말하면, 쿼리가 주어졌을때 key 값에 따라 value값에 접근할 수 있습니다.

이를 파이썬 코드로 구현하면 다음과 같습니다.

In [1]:
dic = {"computer": 9, "dog": 2, "cat": 3}

def key_value_func(query):
    weights = []
    
    for key in dic.keys():
        weights += [is_same(key, query)]
        
    weight_sum = sum(weights)
    
    for i, w in enumerate(weights):
        weights[i] = weights[i] / weight_sum
        
    answer = 0
    
    for weight, value in zip(weights, dic.values()):
        answer += weight * value
        
    return answer


def is_same(key, query):
    if key == query:
        return 1.
    
    else:
        return 0.

순차적으로 dic 변수 내부의 key 값들과 query 값들을 비교하여 key가 같을 경우에는 weights 변수에 1.0을 더하고, 다를 경우에는 0을 더합니다. 그리고 weights를 weights의 총합으로 나누어 그 합이 1이 되도록 만들어줍니다. 다시 dic 내부의 value 값들과 weights의 값에 대해 곱하여 더해줍니다.

### 연속적인 key-value 함수

더 발전시켜서 is_same 함수 대신 다른 함수를 써보면 어떻게 될까요? key와 query 사이의 유사도를 반환하면 how_similar라는 가상 함수가 있다고 가정해봅시다.

<br></br>
```python
>>> query = "puppy"
>>> how_similar("computer", query)
0.1
>>> how_similar("dog", query)
0.9
>>> how_similar("cat", query)
0.7
```
<br></br>


해담 함수에 puppy라는 단어를 테스트했더니 앞에서와 같은 값들을 반환했다고 합시다. 그럼 다음과 같이 실행됩니다.

<br></br>
```python
>>> query = "puppy"
>>> key_value_func(query)
2.823 ## 0.1 / (0.9 + 0.7 + 0.1) * 9 + 0.9 / (0.9 + 0.7 + 0.1) * 2 + 0.7 / (0.9 + 0.7 + 0.1) * 3
```
<br></br>

2.823이라는 값이 나왔습니다.

강아지와 고양이, 그리고 컴퓨터의 유사의 비율에 따른 dic의 값의 비율을 지녔다. 라고 볼수 있습니다. is_same 함수를 쓸때는 두 값이 같은지 if문을 통해 검사하고 값을 할당했으므로, 0과 1로만 이루어진 불연속적인 값이였습니다. 하지만 how_similar를 사용하여 0과 1사이의 연속적은 값을 weights에 할당하여 key_value_func 함수를 수행합니다.

### 연속적인 key-value 벡터 함수

만약 dic의 value가 100차원의 벡터로 구성되어있다면 어떻게 될까요? Query와 key 모두 벡터를 다룬다면 어떻게 될까요? 단어 임베딩 벡터라면? 그리고 how_similar함수는 이 벡터들간의 코사인 유사도를 반환해주는 함수라면? 마지막으로 dic의 key와 value이 서로 같다면 어떻게 될까요?

이를 위해 다시 가상의 함수를 만들어보겠습니다. word2vec 함수는 단어를 입력으로 받아 그 단어에 해당하는 미리 정해진 단어 임베딩 벡터를 반환해준다고 가정합니다. 그리고 how_similar 함수는 두 벡터간의 코사인 유사도 값을 반환합니다.

In [3]:
def key_value_func(query):
    weights = []
    
    for key in dic.keys():
        weights += [how_similar(key, query)]
        
    weights = softmax(weights)
    answer = 0
    
    for weight, value in zip(weights, dic.values()):
        answer += weight * value
        
    return answer        

이번에는 key_value_func는 그 값을 받아 weights에 저장한 후, 모든 weights의 값이 채워지면 softmax 함수를 취할 것입니다. 여기서 `softmax`는 weights의 합의 크기를 1로 고정시키는 정규화 역할을 합니다. 따라서 유사도의 총합에서 차지하는 비율만큼 weights의 값이 채워질것입니다.

<br></br>
```python
>>> len(word2vec("computer"))
100

>>> word2vec("dog")
[0.1, 0.3, -0.7, ...]
 
>>> word2vec("cat")
[0.15, 0.2, -0.3, ...]
  
>>> dic = {word2vec("computer"): word2vec("computer"),
           word2vec("dog"): word2vec("dog"),
           word2vec("cat"): word2vec("cat")}
  
  
>>> query = "puppy"
>>> answer = key_value_func(word2vec(query))         
```
<br></br>

answer의 값에는 어떤 벡터 값이 들어있습니다. 그 벡터는 puppy벡터와, dog, computer, cat 벡터들의 코사인 유사도에 따라 값이 정해집니다. 즉, 이 함수는 query와 비슷한 key를 찾아서 비슷한 정도에 따라 weight를 정하고, 각 keyu의 value를 weight 값 만큼 가져와서 모두 더하는 것입니다.

이것이 바로 **어텐션**의 원리입니다.

### 기계번역에서의 어텐션

번역에서 어텐션은 어떻게 작용할까요? 번역 과정에서는 인코더의 각 time-step별 출력을 key와 value로 하고, 현재 time-step의 디코더 출력을 query로 삼아 어텐션을 계산합니다

<br></br>

|항목|구성|
|---|---|
|query|현재 time-step의 디코더의 출력|
|key|각 time-step별 인코더의 출력|
|value|각 time-step별 인코더의 출력|

<br></br>

<br></br>
```python
context_vector = attention(query = decoder_output,
                           keys = encoder_outputs,
                           values = encoder_outputs)
```
<br></br>

어텐션을 추가한 seq2seq이 수식에는 다음과 같은 부분이 추가됩니다.

<br></br>
$$
w = softmax(h_t^{tgt}WH^{srcT}) \\
c = wH^{src}\ \text{and c is a context vector} \\
\tilde{h_t^{tgt}} = tanh(linear_{2xhs\to hs}([h_t^{tgt};c])) \\
\hat{y_t} = softmax(linear_{hs\to |V_{tgt}|})(\tilde{h_t^{tgt}})
$$
<br></br>

원하는 정보를 어텐션을 통해 인코더에서 획득한 후, 해당 정보를 디코더의 출력과 이어붙여 `tanh`를 취한 후, `softmax` 계산을 통해 time-step의 입력이 되는 $\hat{y_t}$를 구합니다.

<br></br>
![](./images/10-3-5-attention.jpg)
<br></br>

#### 선형 변환

이때 각 입력 파라미터를 추상적으로 예상해본다면 다음과 같습니다. 물론 실제 신경망 내부의 값은 이보다 훨씬 복잡하고 해석할 수 없는 값으로 채워져있습니다.

<br></br>

|항목|의미|
|---|---|
|decoder_output|현재 time-step까지 번역된 타깃 언어의 단어 또는 문장, 의미|
|encoder_outputs|각 time-step에서의 소스 언어의 단어 또는 문장, 의미|

<br></br>

신경망 내부의 각 차원들은 숨겨진 특징 값이므로 딱 잘라 정의할 수 없습니다. 하지만 소스 언어와 대상 언어가 다르므로 단순히 벡터 내적을 해주기보다는 소스 언어와 대상 언어 사이에 연결고리를 하나 놓아주어야 할 것입니다. 따라서 두 언어가 각각 임베딩된 잠재 공간이 선형 관계에 있다고 추상적으로 가정하고, 내적 연산을 수행하기 전에 $선형\ 변환^{linear\ transformation}$을 해줍니다. 이 선형 변환을 위한 W 값은 신경망 가중치 파라미터로써, 피드 포워드 및 역전파를 통해 학습하게 됩니다.

왜 어텐션이 필요할까요? 기존의 seq2seq는 인코더와 디코더라는 두개의 RNN으로 이루어져 있습니다. 여기서 문장 임베딩 벡터에 해당하는 인코더 결과 벡터의 정보를 디코더의 은닉 상태 (LSTM의 경우에는 cell state가 추가)로 전달해야 합니다. 그리고 디코더는 인코더로부터 넘겨받은 은닉 상태로부터 문장을 만들어냅니다. 은닉 상태만으로는 문장의 모든 정보를 완벽하게 전달하기 어렵기 때문입니다. 특히 문장이 길어질수록 이 문제는 더 심각해집니다. 따라서 디코더의 time-step마다 현재 디코더의 은닉 상태에 따라 필요한 인코더의 정보에 접근하여 끌어다 쓰겠다는 것입니다.

<br></br>
![](./images/10-3-5-attention2.jpg)
<br></br>

Linear Transformation을 배우는 것 자체가 어텐션이라고 표현해도 과하지 않습니다. 선형 변환 과정을 통해서 디코더의 현재 상태에 따라 필요한 Query를 만들어내고, 인코더의 key와 비교하여 $가중합^{weighted\ sum}$을 구하는 것이기 때문입니다. 즉, 어텐션을 통해 디코더는 인코더에 Query를 날리는 것인데, Query를 잘 날려야 좋은 정답을 얻을 수 있습니다.

예르 들어 어떤 정보에 관해 알고자 할때, 구글에 검색한다면, 검색창에 검색을 잘 하기 위한 query를 만들어 검색 버튼을 누를 것입니다. 그 query의 질에 따라 검색 결과의 품질은 천차만별일 것입니다. 그리고 같은 정보를 알고자 할때도, 쿼리를 만들어내는 능력에 따라 검색 결과를 얻을 수 있느냐 없느냐는 달라질 것입니다. 쿼리를 만들어내는 훈련을 한다면 그들의 검색능력 역시 좋아질 것입니다.

마찬가지로 신경망도 쿼리를 만들어내는 훈련을 하는 것이라고 볼 수 있습니다. 따라서, 현재 디코더의 상태에 따라 필요한 정보가 무엇인지를 스스로 판단하여 linear transformation을 통해 쿼리를 만들어낼 것입니다. 또한 linear transformation을 위한 가중치 파라미터 자체도 한계가 있으므로, 디코더의 상태 자체가 linear transformation이 되어 쿼리가 좋은 형태가 되도록 RNN이 동작할 것입니다.

#### 어텐션 적용 결과

어텐션을 사용하지 않은 seq2seq는 전반적으로 성능이 떨어짐을 알 수 있을뿐만 아니라, 특히 문장이 길어질수록 성능이 더욱 하락함을 알 수 있습니다. 하지만 이에 비해서 어텐션을 사용하면 문장이 길어지더라도 성능이 크게 하락하지 않음을 알 수 있습니다.

<br></br>
![](./images/10-3-5-result.jpg)
<br></br>


### 파이토치 예제 코드

`torch.bmm()`는 batch matrix multiplication (배치행렬곱)을 수행하는 함수로써, 2개 이상의 차원을 지닌 텐서가 주어졌을때, 뒤의 2개 차원에 대해 행렬 곱을 수행하고, 앞의 다른 차원은 미니배치로 취급합니다. 따라서 앞의 차원들은 크기가 같아야하고, 뒤의 2개 차원은 행렬곱을 수행하기 위한 적절한 크기를 지내야합니다.

<br></br>
```python
import torch

## |x| = (batch_size, n, k)
## |y| = (batch_size, k, m)
## |z| = (batch_size, n, m)
z = torch.bmm(x, yh)
```
<br></br>

<br></br>
![](./images/10-3-6-bmm.jpg)
<br></br>

#### Attention Class

선형 변환을 위한 가중치 파라미터를 bias가 없는 선형 계층으로 대체했습니다.

In [5]:
import torch
import torch.nn as nn

class Attention(nn.Module):
    
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        
        self.linear = nn.Linear(hidden_size, hidden_size, bias = False)
        self.softmax = nn.Softmax(dim = -1)
        
    def forward(self, h_src, h_t_tgt, mask = None):
        ## |h_src| = (batch_size, length, hidden_size)
        ## |h_t_tgt| = (batch_size, 1, hidden_size)
        ## |mask| = (batch_size, length)
        
        ## |query| = (batch_size, hidden_size, 1)
        query = self.linear(h_t_tgt.squeeze(1)).unsqueeze(-1)
        
        ## |weight| = (batch_size, length)
        weight = torch.bmm(h_src, query).squeeze(-1)
        
        if mask is None:
            ## Set each weight as -inf, if the mask value equals to 1
            ## Since the softmax operation makes -inf to 0,
            ## masked weights would be set to 0 after softmax operation
            ## This, if the sample is shorter than other samples in mini-batch,
            ## The weight for empty time-step would be set to 0.
            weight.masked_fill_(mask, -float("inf"))
            
        weight = self.softmax(weight)
        
        ## |context_vector| = (batch_size, 1, hidden_size)
        context_vector = torch.bmm(weight.unsqueeze(1), h_src)
        
        return context_vector